In [1]:
import pandas as pd
import sqlite3
from utils import lire_requete_sql, executer_requete_sql, nettoyer_dataframe


# Chemin vers le fichier de requête SQL
chemin_requete_sql = '../sql_queries/request_model.sql'

# Connexion à la base de données et exécution de la requête
df_reviews = executer_requete_sql(lire_requete_sql(chemin_requete_sql), "../olist.db")

# Netoyage du DataFrame df_reviews
df_reviews = nettoyer_dataframe(df_reviews)

In [2]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29493 entries, 0 to 29494
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   order_id                  29493 non-null  object
 1   customer_zip_code_prefix  29493 non-null  object
 2   customer_city             29493 non-null  object
 3   customer_state            29493 non-null  object
 4   delivery_time_hours       29493 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.4+ MB


In [3]:
df_reviews.head()

,order_id,customer_zip_code_prefix,customer_city,customer_state,delivery_time_hours
0,e481f51cbdc54678b7cc49136f2d6af7,03149,sao paulo,SP,202
1,949d5b44dbf5de918fe9c16f97b45f8a,59296,sao goncalo do amarante,RN,317
2,a4591c265e18cb1dcee52889e2d8acc3,86320,congonhinhas,PR,397
3,e69bfb5eb88e0ed6a785585b27e16dbf,18075,sorocaba,SP,437
4,34513ce0c4fab462a55830c0989c7edb,04278,sao paulo,SP,138


## LinearRegression

In [18]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
from utils import train_linear_regression_with_mlflow

train_linear_regression_with_mlflow(df_reviews)

ImportError: cannot import name 'train_linear_regression_with_mlflow' from 'utils' (/home/utilisateur/Documents/Projets/modele API Charles/ml-model-api-template-devops/model/utils.py)

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import mlflow
import mlflow.sklearn

# Votre DataFrame df_reviews doit contenir les colonnes nécessaires pour l'entraînement du modèle
features = ['customer_zip_code_prefix', 'customer_city', 'customer_state']
target = 'delivery_time_hours'

X = df_reviews[features]
y = df_reviews[target]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construction du pipeline de prétraitement
numeric_features = []  # Aucune caractéristique numérique dans ce cas

categorical_features = ['customer_zip_code_prefix', 'customer_city', 'customer_state']

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

# Création du pipeline complet avec le modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Initialisation de MLflow
mlflow_tracking_uri = "http://127.0.0.1:5000"
experiment_name = "olist_experiment"
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment(experiment_name)

# Enregistrement des paramètres et métriques avec MLflow
with mlflow.start_run():
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = model.predict(X_test)

    # Calcul des métriques
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    # Enregistrement des métriques dans MLflow
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    # Enregistrement du modèle dans MLflow
    mlflow.sklearn.log_model(model, "model")
    
    # Affichage des métriques
    print("Root Mean Squared Error (RMSE):", rmse)
    print("Coefficient de détermination (R^2) sur les données de test:", r2)


/home/utilisateur/Documents/Projets/modele API Charles/ml-model-api-template-devops/env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Root Mean Squared Error (RMSE): 216.02741192561237
Coefficient de détermination (R^2) sur les données de test: -0.03570939594822797


## RandomForestRegressor

In [7]:
# import pickle

# # Enregistrement du modèle
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [8]:
# # Chargement du modèle à partir du fichier pickle
# with open('modele.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

In [9]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # Sélection des colonnes numériques et catégorielles
# numeric_features = ['delivery_time_hours']  # Votre cible
# categorical_features = ['customer_zip_code_prefix', 'customer_city', 'customer_state']

# # Encodage one-hot des caractéristiques catégorielles
# X_cat = pd.get_dummies(df_reviews[categorical_features], drop_first=True)

# # Sélection des fonctionnalités et de la cible
# X = pd.concat([X_cat, df_reviews[numeric_features]], axis=1)
# y = df_reviews['delivery_time_hours']

# # Séparation des données d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Création et entraînement du modèle RandomForest
# model = RandomForestRegressor()
# model.fit(X_train, y_train)

# # Prédiction sur les données de test
# y_pred = model.predict(X_test)

# # Calcul de l'erreur quadratique moyenne (RMSE)
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print("Root Mean Squared Error (RMSE):", rmse)

# # Calcul du score R^2 sur les données de test
# r2 = r2_score(y_test, y_pred)
# print("Coefficient de détermination (R^2) sur les données de test:", r2)

# # Root Mean Squared Error (RMSE): 0.6768629173204135
# # Coefficient de détermination (R^2) sur les données de test: 0.9999912509174399